In [ ]:
from pathlib import Path
from refAV.paths import *
from refAV.dataset_conversion import separate_scenario_mining_annotations, pickle_to_feather

sm_val_feather = Path('av2_sm_downloads/scenario_mining_val_annotations.feather')
separate_scenario_mining_annotations(sm_val_feather, SM_DATA_DIR)
pickle_to_feather()